In [2]:
import os
import sys

current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [3]:
from loguru import logger

In [4]:
import os
import uuid
import json
import pickle
import hashlib
import argparse
from loguru import logger
from lean_dojo import Theorem
from typing import List, Tuple, Optional
from lean_dojo import LeanGitRepo, Theorem, Pos, is_available_in_cache

from common import set_logger
from prover.proof_search import Status, DistributedProver

[2024-05-20 06:06:21,213] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/ubuntu/miniconda3/envs/dojo/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


In [5]:
from evaluate import _get_theorems
from common import set_logger
from prover.proof_search import Status, DistributedProver

In [6]:
data_path = "../data/leandojo_benchmark_4/random"
split = "val"
file_path = None
full_name = None
name_filter = None
num_theorems = None

repo, theorems, positions = _get_theorems(data_path, 
                                          split, 
                                          file_path, 
                                          full_name, 
                                          name_filter, 
                                          num_theorems)

total_theorems = len(theorems)
print(f"Total number of theorems: {total_theorems}")

2024-05-20 06:06:37.348 | INFO     | evaluate:_get_theorems_from_files:121 - Starting from 0th theorem named Affine.Triangle.circumsphere_eq_of_dist_of_oangle!
2024-05-20 06:06:37.349 | INFO     | evaluate:_get_theorems_from_files:127 - 2000 theorems loaded from ../data/leandojo_benchmark_4/random


Total number of theorems: 2000


In [7]:
#CUDA_VISIBLE_DEVICES=1 python prover/evaluate.py     --data-path data/leandojo_benchmark_4/random/     --ckpt_path leandojo-pl-ckpts/generator_random.ckpt     --split test     --num-workers 5     --num-gpus 1     --num-theorems 1 --start-ind 10

In [8]:
from generator.model import RetrievalAugmentedGenerator, FixedTacticGenerator
import torch

In [60]:
joint_ckpt_path = '/home/ubuntu/dojo/github/reprover-test/lightning_logs/joint_ckpt/reprover-joint/s3upj4dw/checkpoints/checkpoint-epoch=00-step=10000-loss_val=1000.0000-loss_train=0.2334.ckpt'
goal_ckpt_path = '/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal/reprover-goal/lkiucv18/checkpoints/checkpoint-epoch=00-step=300-loss_val=0.0000-loss_train=0.1045-timestamp=0.ckpt'
tactic_ckpt_path = '/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal_driven_tactic_ckpt/reprover-goal_driven_tactic/abmoy18k/checkpoints/checkpoint-epoch=01-step=100000-loss_val=0.2866-loss_train=0.1084.ckpt'
tactic_orig_path = '/home/ubuntu/dojo/github/reprover-test/leandojo-pl-ckpts/generator_random.ckpt'

In [29]:
tactic_ckpt_path

'/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal_driven_tactic_ckpt/reprover-goal_driven_tactic/abmoy18k/checkpoints/checkpoint-epoch=01-step=100000-loss_val=0.2866-loss_train=0.1084.ckpt'

In [30]:
goal_ckpt_path

'/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal/reprover-goal/95eb7gqd/checkpoints/checkpoint-00-150-0.0000-0.1250.ckpt'

In [61]:
tac_gen_orig = RetrievalAugmentedGenerator.load(
                ckpt_path=tactic_orig_path, 
                device=torch.device("cuda:3"), 
                freeze=True
            )

2024-05-20 07:47:12.975 | INFO     | generator.model:load:157 - Loading checkpoint from /home/ubuntu/dojo/github/reprover-test/leandojo-pl-ckpts/generator_random.ckpt.


Processing zero checkpoint '/home/ubuntu/dojo/github/reprover-test/leandojo-pl-ckpts/generator_random.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.0
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmpuaoq5m0b/lightning.cpkt


2024-05-20 07:47:25.587 | INFO     | generator.model:__init__:130 - Without retrieval
/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
tac_gen = RetrievalAugmentedGenerator.load(
                ckpt_path=tactic_ckpt_path, 
                device=torch.device("cuda:0"), 
                freeze=True
            )

2024-05-20 06:20:21.744 | INFO     | generator.model:load:157 - Loading checkpoint from /home/ubuntu/dojo/github/reprover-test/lightning_logs/goal_driven_tactic_ckpt/reprover-goal_driven_tactic/abmoy18k/checkpoints/checkpoint-epoch=01-step=100000-loss_val=0.2866-loss_train=0.1084.ckpt.


Processing zero checkpoint '/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal_driven_tactic_ckpt/reprover-goal_driven_tactic/abmoy18k/checkpoints/checkpoint-epoch=01-step=100000-loss_val=0.2866-loss_train=0.1084.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.2
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmph9rbhjti/lightning.cpkt


2024-05-20 06:20:35.464 | INFO     | generator.model:__init__:130 - Without retrieval
/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
theorem = theorems[0]
positions[0]

(299, 1)

In [11]:
from lean_dojo import *

In [14]:
with Dojo(theorem) as (dojo, init_state):
    init_state = init_state

2024-05-20 06:46:46.968 | WARNING  | lean_dojo.interaction.dojo:__init__:162 - Using Lean 4 without a hard timeout may hang indefinitely.


In [47]:
def format_goal_driven_ts(ts, ts_) -> str:
    return f"[CURRENT GOAL]\n{ts}\n[TARGET GOAL]\n{ts_}\n[PROOFSTEP]\n"

def formal_goal(ts) -> str:
    return f"[CURRENT GOAL]\n{ts}\n[TARGET GOAL]\n"

def format_joint_ts(ts) -> str:
    return f"[CURRENT GOAL]\n{ts}\n"

In [48]:
ts = init_state.pp
ts_ = 'no goals'
goal_driven_ts = format_goal_driven_ts(ts, ts_)
goal = formal_goal(ts)
joint_ts = format_joint_ts(ts)

In [36]:
print(goal_driven_ts)

[CURRENT GOAL]
V : Type u_1
P : Type u_2
inst✝⁴ : NormedAddCommGroup V
inst✝³ : InnerProductSpace ℝ V
inst✝² : MetricSpace P
inst✝¹ : NormedAddTorsor V P
hd2 : Fact (finrank ℝ V = 2)
inst✝ : Module.Oriented ℝ V (Fin 2)
t : Triangle ℝ P
i₁ i₂ i₃ : Fin 3
h₁₂ : i₁ ≠ i₂
h₁₃ : i₁ ≠ i₃
h₂₃ : i₂ ≠ i₃
⊢ Simplex.circumsphere t =
    {
      center :=
        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •
            (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ
          midpoint ℝ (t.points i₁) (t.points i₃),
      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }
[TARGET GOAL]
no goals
[PROOFSTEP]



In [37]:
print(goal)

[CURRENT GOAL]
V : Type u_1
P : Type u_2
inst✝⁴ : NormedAddCommGroup V
inst✝³ : InnerProductSpace ℝ V
inst✝² : MetricSpace P
inst✝¹ : NormedAddTorsor V P
hd2 : Fact (finrank ℝ V = 2)
inst✝ : Module.Oriented ℝ V (Fin 2)
t : Triangle ℝ P
i₁ i₂ i₃ : Fin 3
h₁₂ : i₁ ≠ i₂
h₁₃ : i₁ ≠ i₃
h₂₃ : i₂ ≠ i₃
⊢ Simplex.circumsphere t =
    {
      center :=
        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •
            (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ
          midpoint ℝ (t.points i₁) (t.points i₃),
      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }
[TARGET GOAL]



In [49]:
print(joint_ts)

[CURRENT GOAL]
V : Type u_1
P : Type u_2
inst✝⁴ : NormedAddCommGroup V
inst✝³ : InnerProductSpace ℝ V
inst✝² : MetricSpace P
inst✝¹ : NormedAddTorsor V P
hd2 : Fact (finrank ℝ V = 2)
inst✝ : Module.Oriented ℝ V (Fin 2)
t : Triangle ℝ P
i₁ i₂ i₃ : Fin 3
h₁₂ : i₁ ≠ i₂
h₁₃ : i₁ ≠ i₃
h₂₃ : i₂ ≠ i₃
⊢ Simplex.circumsphere t =
    {
      center :=
        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •
            (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ
          midpoint ℝ (t.points i₁) (t.points i₃),
      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }



In [38]:
tac_gen.gen_type

'goal_driven_tactic'

In [39]:
def generate_tactics(ts: str, theorem, tac_gen, num_sampled_tactics=5):
    import time
    t0 = time.monotonic()

    path = str(theorem.file_path)

    suggestions = tac_gen.generate(
        inputs=ts,  # TODO: NEED TO MODIFY THIS TO ADAP DIFFERENT INPUT PATTERNS
        file_path=path,
        theorem_full_name=theorem.full_name,
        theorem_pos=positions[0],
        num_samples=num_sampled_tactics,
    )

    # self.actor_time += time.monotonic() - t0

    logger.debug(f"Tactic suggestions: {suggestions}")
    return suggestions

In [56]:
def generate_tactics(ts: str, theorem, tac_gen, num_sampled_tactics=5):
    import time
    t0 = time.monotonic()

    path = str(theorem.file_path)
    
    if tac_gen.gen_type == 'default':
        ts = ts 
    elif tac_gen.gen_type == 'goal_driven':
        ts = format_goal_driven_ts(ts)
    elif tac_gen.gen_type == 'joint':
        ts = format_joint_ts(ts)

    suggestions = tac_gen.generate(
        inputs=ts,  # TODO: NEED TO MODIFY THIS TO ADAP DIFFERENT INPUT PATTERNS
        file_path=path,
        theorem_full_name=theorem.full_name,
        theorem_pos=positions[0],
        num_samples=num_sampled_tactics,
    )

    # self.actor_time += time.monotonic() - t0

    logger.debug(f"Tactic suggestions: {suggestions}")
    return suggestions

In [40]:
def generate_goal(ts: str, theorem, goal_gen, num_sampled_tactics=5):
    import time
    t0 = time.monotonic()
    
    assert goal_gen.gen_type == 'goal'

    path = str(theorem.file_path)
    current_goal = formal_goal(ts)

    suggestions = goal_gen.generate(
        inputs=current_goal,  # TODO: NEED TO MODIFY THIS TO ADAP DIFFERENT INPUT PATTERNS
        file_path=path,
        theorem_full_name=theorem.full_name,
        theorem_pos=positions[0],
        num_samples=num_sampled_tactics,
    )

    # self.actor_time += time.monotonic() - t0

    logger.debug(f"Goal suggestions: {suggestions}")
    return suggestions

In [20]:
from lean_dojo import *

In [26]:
k = generate_tactics(goal_driven_ts, theorem, tac_gen, num_sampled_tactics=5)

In [27]:
k

[('rfl', -3.8248703479766846),
 ('simp [EuclideanGeometry.circumsphere]', -5.000375747680664),
 ('simp [EuclideanGeometry.circumsphere, h₁₂, h₁₃, h₂₃]', -6.02429723739624),
 ('simp [EuclideanGeometry.circumsphere, EuclideanGeometry.circumsphere]',
  -6.810550689697266),
 ('rw [EuclideanGeometry.circumsphere_eq_circumsphere h₁₂ h₁₃ h₂₃]',
  -7.6300458908081055)]

In [32]:
goal_gen = RetrievalAugmentedGenerator.load(
                ckpt_path=goal_ckpt_path, 
                device=torch.device("cuda:1"), 
                freeze=True
            )

2024-05-20 07:19:01.400 | INFO     | generator.model:load:157 - Loading checkpoint from /home/ubuntu/dojo/github/reprover-test/lightning_logs/goal/reprover-goal/lkiucv18/checkpoints/checkpoint-epoch=00-step=300-loss_val=0.0000-loss_train=0.1045-timestamp=0.ckpt.


Processing zero checkpoint '/home/ubuntu/dojo/github/reprover-test/lightning_logs/goal/reprover-goal/lkiucv18/checkpoints/checkpoint-epoch=00-step=300-loss_val=0.0000-loss_train=0.1045-timestamp=0.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.2
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmp1jjyoakh/lightning.cpkt


2024-05-20 07:19:17.146 | INFO     | generator.model:__init__:130 - Without retrieval
/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [53]:
joint_gen = RetrievalAugmentedGenerator.load(
                ckpt_path=joint_ckpt_path, 
                device=torch.device("cuda:2"), 
                freeze=True
            )

2024-05-20 07:37:55.658 | INFO     | generator.model:load:157 - Loading checkpoint from /home/ubuntu/dojo/github/reprover-test/lightning_logs/joint_ckpt/reprover-joint/s3upj4dw/checkpoints/checkpoint-epoch=00-step=10000-loss_val=1000.0000-loss_train=0.2334.ckpt.


Processing zero checkpoint '/home/ubuntu/dojo/github/reprover-test/lightning_logs/joint_ckpt/reprover-joint/s3upj4dw/checkpoints/checkpoint-epoch=00-step=10000-loss_val=1000.0000-loss_train=0.2334.ckpt/checkpoint'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.14.2
Reconstructed fp32 state dict with 170 params 299637760 elements
Saving fp32 state dict to /tmp/tmp_9nlea_0/lightning.cpkt


2024-05-20 07:38:08.384 | INFO     | generator.model:__init__:130 - Without retrieval
/home/ubuntu/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [57]:
joint_output = generate_tactics(ts, theorem, joint_gen, num_sampled_tactics=5)

In [41]:
target_goals = generate_goal(ts, theorem, goal_gen, num_sampled_tactics=5)

In [58]:
joint_output

[('[TARGET GOAL]\nno goals\n[PROOFSTEP]\nsimp [Orientation.rotation]\n',
  -7.7157206535339355),
 ('[TARGET GOAL]\nV : Type u_1\nP : Type u_2\ninst✝⁴ : NormedAddCommGroup V\ninst✝³ : InnerProductSpace ℝ V\ninst✝² : MetricSpace P\ninst✝¹ : NormedAddTorsor V P\nhd2 : Fact (finrank ℝ V = 2)\ninst✝ : Module.Oriented ℝ V (Fin 2)\nt : Triangle ℝ P\ni₁ i₂ i₃ : Fin 3\nh₁₂ : i₁ ≠ i₂\nh₁₃ : i₁ ≠ i₃\nh₂₃ : i₂ ≠ i₃\n⊢ Simplex.circumsphere t =\n    {\n      center :=\n        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •\n           (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ\n          midpoint ℝ (t.points i₁) (t.points i₁) (t.points i₃),\n      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }\n[PROOFSTEP]\nrw [Orientation.rotation_rotation, Orientation.rotation_rotation]\n',
  -21.866575241088867),
 ('[TARGET GOAL]\nV : Type u_1\nP : Type u_2\ninst✝⁴ : NormedAddCommGroup V\ninst✝³ : I

In [70]:
joint_output[0][0].split('[PROOFSTEP]\n')[-1].strip()

'simp [Orientation.rotation]'

In [67]:
# Example string
example_string = """[TARGET GOAL]
{ex['target_state']}
[PROOFSTEP]
{ex['tactic']}"""

# Find the index of the '[PROOFSTEP]\n' marker
proofstep_index = example_string.find('[PROOFSTEP]\n')

# Extract the part of the string after the '[PROOFSTEP]\n' marker
if proofstep_index != -1:
    result = example_string[proofstep_index + len('[PROOFSTEP]\n'):]
else:
    result = None  # Handle the case where the marker is not found

print(result)


{ex['tactic']}


In [43]:
current_goal = formal_goal(ts)
print(current_goal)

[CURRENT GOAL]
V : Type u_1
P : Type u_2
inst✝⁴ : NormedAddCommGroup V
inst✝³ : InnerProductSpace ℝ V
inst✝² : MetricSpace P
inst✝¹ : NormedAddTorsor V P
hd2 : Fact (finrank ℝ V = 2)
inst✝ : Module.Oriented ℝ V (Fin 2)
t : Triangle ℝ P
i₁ i₂ i₃ : Fin 3
h₁₂ : i₁ ≠ i₂
h₁₃ : i₁ ≠ i₃
h₂₃ : i₂ ≠ i₃
⊢ Simplex.circumsphere t =
    {
      center :=
        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •
            (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ
          midpoint ℝ (t.points i₁) (t.points i₃),
      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }
[TARGET GOAL]



In [45]:
len(target_goals[-1])

2

In [46]:
target_goals[-1][0]

'case intro\nV : Type u_1\nP : Type u_2\ninst✝⁴ : NormedAddCommGroup V\ninst✝³ : InnerProductSpace ℝ V\ninst✝² : MetricSpace P\ninst✝¹ : NormedAddTorsor V P\nhd2 : Fact (finrank ℝ V = 2)\ninst✝ : Module.Oriented ℝ V (Fin 2)\nt : Triangle ℝ P\ni₁ i₂ i₃ : Fin 3\nh₁₂ : i₁ ≠ i₂\nh₁₃ : i₁ ≠ i₃\nh₂₃ : i₂ ≠ i₃\n⊢ Simplex.circumsphere t =\n    {\n      center :=\n        ((Real.Angle.tan (∡ (t.points i₁) (t.points i₂) (t.points i₃)))⁻¹ / 2) •\n            (Orientation.rotation o ↑(π / 2)) (t.points i₃ -ᵥ t.points i₁) +ᵥ\n          midpoint ℝ (t.points i₁) (t.points i₃),\n      radius := dist (t.points i₁) (t.points i₃) / |Real.Angle.sin (∡ (t.points i₁) (t.points i₂) (t.points i₃))| / 2 }'

In [62]:
tactic_orig = generate_tactics(ts, theorem, tac_gen_orig, num_sampled_tactics=5)

In [63]:
tactic_orig

[('fin_cases i₁ <;> fin_cases i₂ <;> fin_cases i₂ <;> fin_cases i₃ <;> simp (config := {decide := Bool.true}) at h₁₂ h₁₃ h₂₃ ⊢',
  -3.765130043029785),
 ('clear h₁₂ h₁₃', -3.943025827407837),
 ('fin_cases i₁ <;> fin_cases i₂ <;> fin_cases i₃ <;> simp (config := {decide := Bool.true}) at h₁₂ h₁₃ h₂₃ ⊢',
  -3.9873223304748535),
 ('fin_cases i₁ <;> fin_cases i₃ <;> simp (config := {decide := Bool.true}) at h₁₂ h₁₃ h₂₃ ⊢',
  -4.106309413909912),
 ('clear h₁₂ h₁₃ h₂₃', -4.790125846862793)]

In [71]:
joint_output[0][0].split('[PROOFSTEP]\n')[-1].strip()

'simp [Orientation.rotation]'

In [72]:
tactic_orig[0][0]

'fin_cases i₁ <;> fin_cases i₂ <;> fin_cases i₂ <;> fin_cases i₃ <;> simp (config := {decide := Bool.true}) at h₁₂ h₁₃ h₂₃ ⊢'